<a href="https://colab.research.google.com/github/withlionbuddha/learning.ai/blob/ground/PseudoLabeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

의사 레이블(Pseudo-label)

---
의사 레이블링은 일반적으로 레이블이 일부만 주어졌을 때, 나머지 데이터를 예측하여 성능을 개선하는데 사용됩니다



In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

In [3]:
import os
import shutil
# MNIST 데이터셋이 저장된 경로
cache_dir = os.path.expanduser('~/.keras/datasets/mnist.npz')

# 캐시 파일 삭제
if os.path.exists(cache_dir):
    os.remove(cache_dir)
    print("Cached MNIST data deleted.")


* mnist은 손으로 쓴 숫자(0-9) 이미지를 모아둔 데이터셋
* train_input은 입력용 훈련이미지데이터
* train_label은 train_input의 정답 데이터
* test_input은 입력용 시험이미지데이터
* test_label은 test_input의 정답 데이터
* 0~255 사이의 정수 값(픽셀 값)으로 이루어진 배열


In [4]:
# MNIST 데이터셋 불러오기
(train_input, train_label), (test_input, test_label) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
print(f"train_input shape: {train_input.shape}")
print(f"axis=0 (batch of images): {train_input.shape[0]}")
print(f"axis=1 (height of each image): {train_input.shape[1]}")
print(f"axis=2 (width of each image): {train_input.shape[2]}")
print(f"train_label shape: {train_label.shape}")

print(f"-----------------------------------")
print(f"test_input shape: {test_input.shape}")
print(f"test_label shape: {test_label.shape}")

train_input shape: (60000, 28, 28)
axis=0 (batch of images): 60000
axis=1 (height of each image): 28
axis=2 (width of each image): 28
train_label shape: (60000,)
-----------------------------------
test_input shape: (10000, 28, 28)
test_label shape: (10000,)



*  입력값(train_expand_input, test_input)은 astype("float32")를 사용하여 데이터를 32비트 부동 소수점 형식으로 변환합니다. 이는 신경망 모델이 실수 값을 처리하는 데 더 적합하기 때문입니다.


* 입력값(train_expand_input, test_input)을 각 이미지의 픽셀 값(0~255)을 255.0으로 나눠서 0과 1 사이의 값으로 스케일링합니다.




In [7]:
# 데이터 전처리
train_expand_input = np.expand_dims(train_input, axis=-1)
print(f"train_expand_input shape: {train_expand_input.shape}")

train_expand_input = train_expand_input.astype("float32") / 255.0
test_input = test_input.astype("float32") / 255.0

train_expand_input shape: (60000, 28, 28, 1)



* numpy.ndarray 는 NumPy 라이브러리의 다차원 배열 객체로, 효율적인 수치 연산

In [10]:
type(train_expand_input)

numpy.ndarray

데이터 분할 , 데이터 슬라이싱

train_expand_input[:num_labeled]은 train_expand_input의 6000개의 images 중에서 1000개의 images를 분할하여 input_labeled에 할당합니다.

In [21]:
# 레이블이 있는 데이터를 일부만 사용하고 나머지는 의사 레이블로 처리
# 여기서는 예시로 train_label의 앞부분만 사용
num_labeled = 1000
train_input_labeled = train_expand_input[:num_labeled]
train_label_labeled = train_label[:num_labeled]

input_labeled의 형태는

In [22]:
print(f"train_input_labeled shape: {train_input_labeled.shape}")
print(f"train_label_labeled shape: {train_label_labeled.shape}")

train_input_labeled shape: (1000, 28, 28, 1)
train_label_labeled shape: (1000,)


In [23]:
test_input_unlabeled = test_input[num_labeled:]
test_label_unlabeled = test_label[num_labeled:]  # 원래 라벨은 실제 학습에 사용되지 않음

print(f"test_input_unlabeled shape: {test_input_unlabeled.shape}")
print(f"test_label_unlabeled shape: {test_label_unlabeled.shape}")

test_input_unlabeled shape: (9000, 28, 28)
test_label_unlabeled shape: (9000,)


In [19]:
# 간단한 모델 구성
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
# 먼저 라벨이 있는 데이터로 학습
model.fit(train_input_labeled , train_label_labeled , epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.3714 - loss: 71.8700 - val_accuracy: 0.6950 - val_loss: 20.6599
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8336 - loss: 6.0208 - val_accuracy: 0.7900 - val_loss: 13.5004
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9428 - loss: 1.4402 - val_accuracy: 0.7900 - val_loss: 12.4891
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9710 - loss: 0.5413 - val_accuracy: 0.7950 - val_loss: 12.1829
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9871 - loss: 0.0843 - val_accuracy: 0.7800 - val_loss: 11.7318


In [ ]:


# 의사 레이블 생성: 레이블이 없는 데이터를 모델을 통해 예측
pseudo_labels = np.argmax(model.predict(test_input_unlabeled), axis=1)

# 라벨이 없는 데이터에 대해 예측된 의사 레이블로 다시 학습
x_combined = np.concatenate([x_labeled, test_input_unlabeled ])
y_combined = np.concatenate([y_labeled, pseudo_labels])

model.fit(x_combined, y_combined, epochs=5, batch_size=32, validation_split=0.2)

# 테스트 데이터로 성능 확인
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.3915 - loss: 1.9721 - val_accuracy: 0.7850 - val_loss: 1.0300
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8425 - loss: 0.7671 - val_accuracy: 0.8300 - val_loss: 0.6708
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8965 - loss: 0.4253 - val_accuracy: 0.8500 - val_loss: 0.5719
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9302 - loss: 0.3105 - val_accuracy: 0.8650 - val_loss: 0.5236
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9439 - loss: 0.2461 - val_accuracy: 0.8550 - val_loss: 0.5277
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9542 - loss: 0.1605 - val_accuracy: 0.9606 - val_loss: 0.0986
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9663 - loss: 0.0915 - val_accuracy: 0.9671 - val_loss: 0.0796
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9727 - loss: 0.